<a href="https://www.kaggle.com/code/abhayyadav1074/dcgan?scriptVersionId=245099268" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## We will first import the required files. Note: You may have to install the libraries which are not currently installed. e.g.: imageio

In [ ]:
pip install imageio


In [ ]:
import tensorflow as tf
import numpy as np 
import math                     #data normalization
import matplotlib.pyplot as plt #plot images of outputs
import os                       # to read 
import imageio                  # to create anim of outputs
from sklearn import preprocessing #for standardizing the dataset



## Load the Fashion-MNIST dataset

## We are going to use the Fashion-MNIST dataset
## Number of images: 60000
## Dimensions: 28X28X1
## This dataset has objects belonging to 10 classes.They are:
## 0: T-shirt/top, 1: Trouser, 2: Pullover, 3: Dress, 4: Coat, 5: Sandal, 6: Shirt, 7: Sneaker, 8: Bag, 9: Ankle boot

## We will first load the dataset, then visualize some sample images from the dataset

In [ ]:
(x_train, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
print(x_train.shape,y_train.shape)


In [ ]:
#plotting 
for i in range(3):  #we plot 10 images
    plt.subplot(1,3,1+i)   #2 rows, 5 cols, index of the image
    plt.imshow(x_train[i*5], cmap=plt.get_cmap('gray'))
    plt.show()


## Building the Generator Network

### Since the generator will up-sample the noise vector, we will use Conv2DTranspose module from Tensorflow. We create the network which performs the following transforms.

### Input the Noise (100X1) to a Dense Layer (7X7X256) followed by series of up-sampling layers Conv2DTranspose to reach the desired image size (28X28X1)

 

In [ ]:
#Convert the train dataset into a 3D DATASET of stacked 3D images 
image_width = x_train.shape[1];
x_train = np.reshape(x_train, [-1, image_width ,image_width , 1]).astype('float32')


In [ ]:
#Check the shape of the data now
x_train.shape


### We will now standardize the data values. Currently the range of values is from 0 to 255. We may either modify it to be between 0 and 1 or between -1 to 1. If we intend to use the tanh activation function in the last layer, we may use the range -1 to 1 or use the range 0 to 1 for sigmoid activation in the output layer
### Now, let use standardize the values

In [ ]:
# Normalize the images to [-1, 1] [Normalized Value = (value - mean)/mean]
x_train = (x_train - 127.5) / 127.5  


In [ ]:
#Generator Initializations
noise_length = 100


### We will use the Tensorflow functional API to build our network. By default the function considers a noise input size of 100, and image size of 28. Both parameters can be customized when the function is called.
### We manipulate the stride in order to increase the dimensions of the input from 7 to 28 and maniputate the filter count to downsize the output to 1 channel in the last layer.

In [ ]:
def create_generator_model(image_size=28, noise_input=100):
   
   #Create input layer
   input_layer = tf.keras.layers.Input(shape=(noise_input,))
   
   #First upsampling to 7X7X256 
   #Increase dimensions and resize to 3D to feed it to Conv2DTranspose layer
   x = tf.keras.layers.Dense(7 * 7 * 256)(input_layer)
   x = tf.keras.layers.Reshape((7, 7, 256))(x)
   
   #Upscaling 1 : 128 filters, (2,2) stride
   #Input = 7X7X256
   #Output at this stage = 14X14X128
   #Syntax Note: kernel size can be specified as a tuple or integer or list
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('leaky_relu')(x)
   x = tf.keras.layers.Conv2DTranspose(128, (5,5), strides=2, padding='same')(x)
   
   #Upscaling 2
   #Input = 14X14X128
   #Output at this stage = 28X28X64
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('leaky_relu')(x)
   x = tf.keras.layers.Conv2DTranspose(64, kernel_size=[5,5], strides=2, padding='same')(x)
   
   #Upscaling 3
   #Input = 28X28X64
   #Output at this stage = 28X28X32
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('leaky_relu')(x)
   x = tf.keras.layers.Conv2DTranspose(32, kernel_size=[5,5], strides=1, padding='same')(x)
   
   #Upscaling 3
   #Input = 28X28X32
   #Output at this stage = 28X28X1
   #Note that we use tanh activation and not Leaky Relu for the last layer. 
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('leaky_relu')(x)
   x = tf.keras.layers.Conv2DTranspose(1, kernel_size=[5,5], strides=1, padding='same')(x)
   
   x = tf.keras.layers.Activation('sigmoid')(x)
   #Create the model    
   gen_network = tf.keras.models.Model(input_layer, x, name='gen_network')
   
   return gen_network


In [ ]:
def create_discriminator_model(desc_data=[28,28,1,]):
   
   #A typical Convolution  network for classification is built  
   disc_input = tf.keras.layers.Input(desc_data)
   #Input Dimension : 28X28X1
   #Output Dimension : 14X14X32
   x = tf.keras.layers.LeakyReLU(alpha=0.2)(disc_input)
   x = tf.keras.layers.Conv2D(32, kernel_size=[5,5], strides=2, padding='same')(x)
   
   #Input Dimension : 14X14X32
   #Output Dimension : 7X7X64
   x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
   x = tf.keras.layers.Conv2D(64, kernel_size=[5,5], strides=2, padding='same')(x)
   
   #Input Dimension : 7X7X64
   #Output Dimension : 4X4X128
   x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
   x = tf.keras.layers.Conv2D(128, kernel_size=[5,5], strides=2, padding='same')(x)
   
   #Input Dimension : 4X4X128
   #Output Dimension : 4X4X256
   x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
   x = tf.keras.layers.Conv2D(256, kernel_size=[5,5], strides=1, padding='same')(x)
   
   #Input Dimension : Flattened(4X4X256)
   #Output Dimension : 1
   #Flatten the output and build an output layer
   x = tf.keras.layers.Flatten()(x)
   x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
   
   #Build Model
   disc_network = tf.keras.models.Model(disc_input, x, name='disc_network')
   
   return disc_network


In [ ]:
generator = create_generator_model()
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
plt.imshow(generated_image[0, :, :, 0], cmap='gray')


In [ ]:
#This code will check the functioning of the descriminator against a sample input. As of this point, 
#the descriminator is untrained.
discriminator = create_discriminator_model()
decision = discriminator(generated_image)
print (decision)


In [ ]:
def build_models():
   
   noise_size = 100
   lr = 2e-4
   decay = 6e-8
   
   #Build Base Discriminator model
   base_discriminator = create_discriminator_model(desc_data=(28,28,1,))
   
   #Define optimizer and compile model
   discriminator = tf.keras.models.Model(inputs=base_discriminator.inputs, 
                                         outputs=base_discriminator.outputs)
   optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)


   discriminator.compile(loss='binary_crossentropy',
                         optimizer=optimizer,
                         metrics=['accuracy'])
   
   #Build Generator model
   generator = create_generator_model(image_size=28, noise_input=noise_size)
   
   #Build Frozen Discriminator
   frozen_discriminator = tf.keras.models.Model(inputs=base_discriminator.inputs, 
                                         outputs=base_discriminator.outputs)
   #Freeze the weights of discriminator during adversarial training
   frozen_discriminator.trainable = False
   #Build Adversarial model
   optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

   #Adversarial = generator + discriminator
   adversarial = tf.keras.models.Model(generator.input, 
                       frozen_discriminator(generator.output))
   
   adversarial.compile(loss='binary_crossentropy',
                       optimizer=optimizer,
                       metrics=['accuracy'])    
   
   return generator, discriminator, adversarial


In [ ]:
def train_gan(generator, discriminator, adversarial, noise_size=100, train_steps=2000, batch_size=32, preview_interval=200):
    import time
    import tensorflow as tf
    import numpy as np

    image_size = 28

    # Load Fashion MNIST dataset
    (x_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

    # Preprocess: reshape and normalize
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1]).astype('float32') / 255.0

    # Create test noise input for visual inspection
    test_noise_input = np.random.uniform(-1.0, 1.0, size=[16, noise_size])

    print(f"Starting training for {train_steps} steps with batch size {batch_size}...\n")
    start_time = time.time()

    for i in range(train_steps):
        # === Train Discriminator ===
        noise_input = np.random.uniform(-1.0, 1.0, size=[batch_size, noise_size])
        fake_images = generator.predict(noise_input, verbose=0)

        real_images = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

        X = np.concatenate((real_images, fake_images))
        y_real = np.ones((batch_size, 1)) * 0.9

        y_fake = np.zeros((batch_size, 1))
        y = np.concatenate((y_real, y_fake))

        d_loss, d_acc = discriminator.train_on_batch(X, y)

        # === Train Adversarial (Generator through combined model) ===
        noise_input = np.random.uniform(-1.0, 1.0, size=[batch_size, noise_size])
        y_adv = np.ones((batch_size, 1))

        a_loss, a_acc = adversarial.train_on_batch(noise_input, y_adv)

        # Print status
        if i % 100 == 0:
            print(f"{i} [D loss: {d_loss:.6f}, acc: {d_acc:.6f}, A loss: {a_loss:.6f}, acc: {a_acc:.6f}]")

        # Save preview images
        if (i + 1) % preview_interval == 0:
            fake_images = generator.predict(test_noise_input, verbose=0)
            plot_images(fake_images, i + 1)

    elapsed = time.time() - start_time
    print(f"\nTraining completed in {elapsed:.2f} seconds.")

    # Save final generator model
    generator.save('fashionmnist_generator_dcgan_fast.h5')


In [ ]:
def plot_images(fake_images, step):
   
   plt.figure(figsize=(2.5,2.5))
   num_images = fake_images.shape[0]
   
   image_size = fake_images.shape[1]
   rows = int(math.sqrt(fake_images.shape[0]))
   
   for i in range(num_images):
       plt.subplot(rows, rows, i + 1)
       image = np.reshape(fake_images[i], [image_size, image_size])
       plt.imshow(image, cmap='gray')
       plt.axis('off')
   plt.show()


In [ ]:
G, D, A = build_models()


In [ ]:
G.summary()


In [ ]:
A.summary()


In [ ]:
D.summary()

In [ ]:
train_gan(G, D, A)


In [ ]:
import glob
import imageio

anim_file = 'dcfashiongan.gif'

filenames = glob.glob(r'D:\Sups\Python\GAN\FASHION_mnist\*.png')
filenames = sorted(filenames)

with imageio.get_writer(anim_file, mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    
    # Append the last image again
    if filenames:
        last_image = imageio.imread(filenames[-1])
        writer.append_data(last_image)
